# 自然言語処理のキソ
NLP;Natural Language Processing

- 本発表では文書同士の類似度の比較方法を例に見ていきます

法政大学　卒業生　湯浅大地

## 自然言語
- 自然言語とは、私たちが普段使っている言葉のこと
- プログラミング言語などの人工的に作られた言語の対比として「自然」と呼ばれる

## 自然言語処理のやりたいこと
- 自然言語を数学的に表現する⇨機械学習、AIに応用することができる

##  自然言語処理の世界に一緒に踏み入れましょう！

# ケーススタディ
一番似ている２人を探せ！！

- Aさん：ステーキが好きです。ハンバーグも好きです。
- Bさん：ステーキがとても好きです。
- Cさん：パスタが好きです。


## どうやって探す？

1.文書を数学的に表現にする
- Bag-Of-Words（Bow）の手法を用いてベクトルで表現する

2.文書の比較
- コサイン類似度の手法を用いる

1.文書を数学的に表現にする
## まずは単語分解！！
専門的には形態素解析といいます。

- 文章を名詞、動詞、形容詞などの品詞別に分解する
- 日本語は英語と違って文章が単語ごとに区切られていない
- 日本語を形態素解析できるソフトウェアとして広く使われているのはMeCab

### MeCab

- 京都大学とNTTの基礎研究所が共同開発したもの
- オープンソースソフトウェアでダウンロード、使用が全て無料
- 内蔵されているIPA辞書という日本語辞書には約40万語の単語が登録されている

### 『ステーキとハンバーグが好きです。 』を単語分解！

In [72]:
# MeCabライブラリをインポート
import MeCab
# パーサーの設定。ChaSenという形態素解析器と互換の出力をする設定にしている。
tagger = MeCab.Tagger('-Ochasen')
# 形態素解析
testNode = tagger.parseToNode("ステーキが好きです。ハンバーグも好きです。")
while testNode:
    print(testNode.feature)
    testNode = testNode.next

BOS/EOS,*,*,*,*,*,*,*,*
名詞,一般,*,*,*,*,ステーキ,ステーキ,ステーキ
助詞,格助詞,一般,*,*,*,が,ガ,ガ
名詞,形容動詞語幹,*,*,*,*,好き,スキ,スキ
助動詞,*,*,*,特殊・デス,基本形,です,デス,デス
記号,句点,*,*,*,*,。,。,。
名詞,一般,*,*,*,*,ハンバーグ,ハンバーグ,ハンバーグ
助詞,係助詞,*,*,*,*,も,モ,モ
名詞,形容動詞語幹,*,*,*,*,好き,スキ,スキ
助動詞,*,*,*,特殊・デス,基本形,です,デス,デス
記号,句点,*,*,*,*,。,。,。
BOS/EOS,*,*,*,*,*,*,*,*


In [73]:
# 形態素解析の関数
def word_tokenaize(doc):
    node = tagger.parseToNode(doc)

    result = []
    while node:
        hinshi = node.feature.split(",")[0]
        if hinshi == '名詞' or hinshi == '動詞':
            result.append(node.feature.split(",")[6])
        node = node.next

    return ' '.join(result)

## 文章を品詞分解したが、品詞には、一人一人の特徴とは関係ないものも含まれている

- 「と」「が」「です」「。」

## このような特徴とは関係のない単語のことをストップワード（Stop Words）と呼ばれる

- 単語を選別することで、意義深い結果が得やすくなるだけでなく、計算負荷を減らすことができる



In [74]:
# ストップワードを除去した後の単語分解の結果
word_tokenaize("ステーキが好きです。ハンバーグも好きです。")

'ステーキ 好き ハンバーグ 好き'

In [75]:
import pandas as pd

In [76]:
df = pd.DataFrame({'x': ["ステーキが好きです。ハンバーグも好きです。", "ステーキがとても好きです。", "パスタが好きです。"]})

In [77]:
df

,x
0,ステーキが好きです。ハンバーグも好きです。
1,ステーキがとても好きです。
2,パスタが好きです。


In [78]:
df['wakati'] = df['x'].apply(word_tokenaize)

### 3人の文章をストップワード除いて単語分解してみる

In [79]:
df

,x,wakati
0,ステーキが好きです。ハンバーグも好きです。,ステーキ 好き ハンバーグ 好き
1,ステーキがとても好きです。,ステーキ 好き
2,パスタが好きです。,パスタ 好き


# やっと！文章を数学的に表現してみる

## Bag-of-Words
- 単語にベクトルに各列を割り当てておいて、出現回数などを要素とすることで文章をベクトル化したもの

In [80]:
from sklearn.feature_extraction.text import CountVectorizer

In [81]:
# Bag-of-Words
vectorizer = CountVectorizer()
bag = vectorizer.fit_transform(df['wakati'])

In [82]:
df_bag = pd.DataFrame(data=bag.toarray(), columns=vectorizer.get_feature_names())

In [83]:
df_bag

,ステーキ,ハンバーグ,パスタ,好き
0,1,1,0,2
1,1,0,0,1
2,0,0,1,1


# ついに！文章同士の類似度を算出

## コサイン類似度
- 算出方法

$$
    cos(a,b) = \frac{\langle a,b\rangle}{\|a\|\|b\|} = \frac{aとbの内積}{aの大きさ、bの大きさ}
$$




- 上の数式で理解しなくともコサインで理解すれば簡単!

|θ|0°|30°|45°|60°|90°|120°|135°|150°|180°|
|:-|:-|:-|:-|:-|:-|:-|:-|:-|:-|
|cosθ|1|$$ \frac{\sqrt3}{2} $$|$$ \frac{1}{\sqrt2} $$|$$ \frac{1}{2} $$|0|$$ -\frac{1}{2} $$|$$ -\frac{1}{\sqrt2} $$|$$ -\frac{\sqrt3}{2} $$|-1|

- コサイン類似度は-1〜1をとりますが、Bag-of-Wordsの手法ではマイナス要素のベクトルが作られないので、0〜1の値の範囲が算出されます


In [84]:
from sklearn.metrics.pairwise import cosine_similarity

In [85]:
cs = cosine_similarity(bag.toarray(),bag.toarray())

In [86]:
df_cs = pd.DataFrame(cs, index=['A','B','C'], columns=['A','B','C'])

# コサイン類似度の算出結果

- 比較文書

    - Aさん：ステーキが好きです。ハンバーグも好きです。
    - Bさん：ステーキがとても好きです。
    - Cさん：パスタが好きです。


In [87]:
df_cs

,A,B,C
A,1.000000,0.866025,0.57735
B,0.866025,1.000000,0.50000
C,0.577350,0.500000,1.00000


# 今後の展望
- コサイン類似度
    - この後発表を控えている宮澤研究を楽しみにしてください。
- トーン分析（感情分析）
    - 極性辞書の単純集計型
    - ナイーブベイズ分類器（機械学習の一つ）でネガポジを推計
    - RNN（リカレントニューラルネットワーク）でネガポジを推計
    - ファイナンスに特化した日本語極性辞書の作成    